In [2]:
import os
import openai
import PyPDF2
import random
import pinecone

from openai import OpenAI

/home/subra/miniconda3/envs/open_ai/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
client = OpenAI()

## Newst OpenAI API UPDATE -> Built-in RAG system

#### Creating a file for the gpt assistant

In [9]:
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("state_of_ai_docs.pdf", "rb"),
  purpose='assistants'
)

#### Creating the gpt assistant

In [10]:
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

#### Creating a thread

To see how this thread is connected to the assistant above

In [11]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What are the main trends in AI?",
      "file_ids": [file.id]
    }
  ]
)

#### Adding messages to the thread above

In [12]:
thread.message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I can't find in the PDF manual how to turn off this device.",
  file_ids=[file.id]
)

#### Runnning the thread

Answer to the above question is the thread and assistant are linked when running the thread

In [20]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

#### Checking status of the thread execution

In [24]:
run=client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run.status

'completed'

#### Results of the thread execution

In [48]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [49]:
## User Questions
[message.content[0].text.value for message in messages.data if message.assistant_id is None][::-1]

['What are the main trends in AI?',
 "I can't find in the PDF manual how to turn off this device."]

In [50]:
## Assistant Responses

[message.content[0].text.value for message in messages.data if message.assistant_id is not None][::-1]

['I have searched the document using terms related to turning off a device, such as "turn off," "power off," and "shutting down," but have not found any relevant information pertaining to your request within the visible portions of the document.\n\nGiven the current state, it appears the instructions on how to turn off the device might be located in a different section that is not captured by the search terms used or is not included in the document. If you could provide more context or specific terms that the manual might use for the process of turning off the device, I could conduct a more targeted search. Otherwise, I can scroll through the document to find related information. Would you like me to do that?']